# 1.2 Fingerprint hashing

Using the previously selected data with the features you found pertinent, you have to:

Implement your minhash function from scratch. No ready-made hash functions are allowed. Read the class material and search the internet if you need to. For reference, it may be practical to look at the description of hash functions in the book.

Process the dataset and add each record to the MinHash. The subtask's goal is to try and map each consumer to its bin; to ensure this works well, be sure you understand how MinHash works and choose a matching threshold to use. Before moving on, experiment with different thresholds, explaining your choice.

In [1]:
import pandas as pd
from tqdm import tqdm as tq
import warnings
import numpy as np
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("/Users/giacomo/Desktop/ADM_HW4/data.csv", sep = '\t')

In [3]:
df

,Unnamed: 0,New_ID,CustGender,CustomerClassAge,Richness,Expenditure
0,0,C1010011F24,F,age_2,richness_7,exp_10
1,1,C1010011M33,M,age_4,richness_9,exp_5
2,2,C1010012M22,M,age_2,richness_6,exp_8
3,3,C1010014F24,F,age_2,richness_7,exp_8
4,4,C1010014M32,M,age_4,richness_9,exp_4
...,...,...,...,...,...,...
1034947,1034947,C9099836M26,M,age_3,richness_9,exp_7
1034948,1034948,C9099877M20,M,age_1,richness_9,exp_4
1034949,1034949,C9099919M23,M,age_2,richness_3,exp_3
1034950,1034950,C9099941M21,M,age_2,richness_7,exp_1


In [4]:
del df['Unnamed: 0']

# 1.2.1 Shingles

First of all we build the shingles from all the unique values per column in the loaded dataset. We ignore the `TransactionID` column because it is not a shingle.

In [5]:
shingles = []
for column_name in df.columns[1:]:
    shingles += sorted(list(df[column_name].unique()))
    
shingles.remove('age_0')

#In order to not aggregate people who are labelled with age_0, corresponding to the Customer DOB with year 1800 
#(nan), we decided to remove age_0 from shingles such that those people will not have any 1 in the shingle matrix.
#For that reason they will not be considered similar to anyone for the age, but only for the other fields.

In [6]:
print(shingles)

['F', 'M', 'age_1', 'age_10', 'age_11', 'age_12', 'age_13', 'age_14', 'age_15', 'age_16', 'age_17', 'age_2', 'age_3', 'age_4', 'age_5', 'age_6', 'age_7', 'age_8', 'age_9', 'richness_0', 'richness_1', 'richness_10', 'richness_2', 'richness_3', 'richness_4', 'richness_5', 'richness_6', 'richness_7', 'richness_8', 'richness_9', 'exp_1', 'exp_10', 'exp_2', 'exp_3', 'exp_4', 'exp_5', 'exp_6', 'exp_7', 'exp_8', 'exp_9']


# 1.2.2 Create Shingle Matrix

First of all we create the function which maps each transaction into a vector of 0/1 based on the shingles. 

In [7]:
def one_hot_vector(data, index):
    """Creates a one hot vector for the row found in the data at the given index based on the shingles.
    
    :args
    data - a pandas dataframe containing the data.
    index - an int which corresponds to the row that will be turned into a one hot vector.
    
    :returns
    a numpy array one hot representation of the row
    """
    
    values = data.loc[index][['CustGender', 'CustomerClassAge', 'Richness', 'Expenditure']].values #extract values
    
    indeces = np.where(values.reshape(values.size, 1) == shingles)[1]  #save indexes
    
    vector = np.zeros(len(shingles), dtype = int)  #initialize vector
    
    vector[indeces] = 1  #substitute 1 in the correct positions
    
    return vector

Example:

In [8]:
print(shingles)

['F', 'M', 'age_1', 'age_10', 'age_11', 'age_12', 'age_13', 'age_14', 'age_15', 'age_16', 'age_17', 'age_2', 'age_3', 'age_4', 'age_5', 'age_6', 'age_7', 'age_8', 'age_9', 'richness_0', 'richness_1', 'richness_10', 'richness_2', 'richness_3', 'richness_4', 'richness_5', 'richness_6', 'richness_7', 'richness_8', 'richness_9', 'exp_1', 'exp_10', 'exp_2', 'exp_3', 'exp_4', 'exp_5', 'exp_6', 'exp_7', 'exp_8', 'exp_9']


In [9]:
df.loc[1]

New_ID              C1010011M33
CustGender                    M
CustomerClassAge          age_4
Richness             richness_9
Expenditure               exp_5
Name: 1, dtype: object

In [10]:
print(one_hot_vector(df, 1))

[0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0
 0 0 0]


In [11]:
len(df)

1034952

Now we can build a sparse matrix with all the encoded transaction. We don't need to insert into the matrix the name of the customerIDs cause we they are linked to the index of the shingle matrix through the index of the column.  

In [12]:
shingle_matrix = np.zeros((1034952, 40), dtype = int)

for i in tq(range(len(df))):
    # Append the one hot vectors as rows
    shingle_matrix[df.index[i]] = one_hot_vector(df, i) 

# We need to transpose because for the shuffling, the Shingles need to be the rows
shingle_matrix = shingle_matrix.T

100%|███████████████████████████████| 1034952/1034952 [13:05<00:00, 1316.87it/s]


In [13]:
%store shingle_matrix

Stored 'shingle_matrix' (ndarray)


In [14]:
shingle_matrix.shape

(40, 1034952)

# 1.2.3 Create the Signature Matrix
From the Shingle Matrix, we will now create the signature matrix by doing the following:
1. Shuffle the rows of the Shingle Matrix.
1. Create a vector where each element corresponds to the index of the row of each column (Shingle) where the first 1 is found.
1. Append this vector to the Signature Matrix.
1. Repeat $n$ times.

The goal of the MinHash is to replace a large set with a smaller "signature" that still preserves the underlying similarity metric.

In [15]:
n_permutations = 20
signature_matrix = np.zeros((20, shingle_matrix.shape[1]), dtype = int)

In [16]:
for i in tq(range(n_permutations)):
    # 1. Shuffle rows
    np.random.shuffle(shingle_matrix)
    
    # 2. Create the vector of indeces where the first 1 is found. np.argmax stops at the first occurrence
    signature_row = np.argmax(shingle_matrix == 1, axis=0) + 1
    
    # 3. Add to signature matrix
    signature_matrix[i] = signature_row

100%|███████████████████████████████████████████| 20/20 [00:33<00:00,  1.69s/it]


In [17]:
signature_matrix

array([[23,  1,  8, ..., 16, 13,  3],
       [ 3,  6,  2, ..., 14,  3,  5],
       [12, 14,  3, ...,  6, 14, 14],
       ...,
       [ 9,  2, 13, ..., 13, 13,  2],
       [ 3,  1,  7, ...,  4,  7, 14],
       [ 3, 22,  1, ...,  5, 10, 30]])

In [18]:
signature_matrix.shape

(20, 1034952)

The index of the column can be referred to the customer ID looking at the index of the initial dataframe: 

In [19]:
df

,New_ID,CustGender,CustomerClassAge,Richness,Expenditure
0,C1010011F24,F,age_2,richness_7,exp_10
1,C1010011M33,M,age_4,richness_9,exp_5
2,C1010012M22,M,age_2,richness_6,exp_8
3,C1010014F24,F,age_2,richness_7,exp_8
4,C1010014M32,M,age_4,richness_9,exp_4
...,...,...,...,...,...
1034947,C9099836M26,M,age_3,richness_9,exp_7
1034948,C9099877M20,M,age_1,richness_9,exp_4
1034949,C9099919M23,M,age_2,richness_3,exp_3
1034950,C9099941M21,M,age_2,richness_7,exp_1


For example the first column of the signature matrix is referred to the customer C1010011F24.

# 1.2.4 Divide Signature Matrix into Bands

The example signature matrix below is divided into $b$ bands of $r$ rows each, and each band is hashed separately. For this example, we are setting band , which means that we will consider any titles with the same first two rows to be similar. The larger we make b the less likely there will be another Paper that matches all of the same permutations.

![signature_matrix_into_bands](https://storage.googleapis.com/lds-media/images/locality-sensitive-hashing-lsh-buckets.width-1200.png)

Ultimately, the size of the bands control the probability that two items with a given Jaccard similarity end up in the same bucket. If the number of bands is larger, you will end up with much smaller sets. For instance, $b = p$, where $p$ is the number of permutations (i.e. rows in the signature matrix) would almost certainly lead to $N$ buckets of only one item because there would be only one item that was perfect similar across every permutation.

In [37]:
b = 4

In [38]:
signature_matrix.T[:, 0:3]

array([[23,  3, 12],
       [ 1,  6, 14],
       [ 8,  2,  3],
       ...,
       [16, 14,  6],
       [13,  3, 14],
       [ 3,  5, 14]])

In order to create the buckets we decide to create a dictionary that will have the sub_vector as keys and the indexes that contains that subvector. These indexes will allow us to substistute the related customer:

In [39]:
indexes = list(range(signature_matrix.shape[1])) #create a list of indexes 

signature_matrix_transposed = signature_matrix.T #transpose the matrix to get subvectors column

cluster = {} #initialize the dictionary containing as keys the subvector and as values the indexes of the customer

for i in tq(range(0, signature_matrix.shape[0], b)):  #iterate over the row with step size equal to bandsize
    
    #take the subvector of dimension i, i+b (band size) from the column
    mini_vectors = signature_matrix_transposed[:, i:i+b] 
    
    # sorts the subvectors associated to the indexes to maintain the relationship with the index of the customers.
    # We use a tuple instead of a list because tuples can be hashable and therefore 
    # usable as keys for dictionaries. In this way we will have the same subvector as neighbors
    
    c = [(i, tuple(v)) for v, i in sorted(zip(mini_vectors.tolist(), indexes))]
    
    curr_vector = c[0][1] #take the subvector from the tuple composed by index and subvector
    
    #Now we have equal subvector as neighbor, so we can iterate over these groups of equal subvector
    for i, v in c:  
        
        if v not in cluster: #if the subvector is not a key in the cluster --> initialize it 
            
            cluster[v] = []
        
        if curr_vector != v: #when the iteration go over the group of equal subvector updtate the current vector
            
            curr_vector = v

        cluster[v].append(i) #append as values the indexes
            

100%|█████████████████████████████████████████████| 5/5 [00:29<00:00,  5.91s/it]


As example we print the first 5 keys -> subvector:

In [40]:
print(list(cluster.keys())[:5])

[(1, 1, 4, 7), (1, 1, 5, 7), (1, 1, 5, 13), (1, 1, 8, 7), (1, 1, 8, 13)]


For example for the subvector 1,1,1,9 we have as values the indexes of the Customers that have been mapped in the same bucket:

In [47]:
cluster[(1, 1, 5, 13)]

[141,
 803,
 2674,
 4190,
 6203,
 6462,
 6689,
 6997,
 7846,
 7867,
 7977,
 8009,
 8639,
 11495,
 13888,
 14026,
 14353,
 14855,
 15584,
 15916,
 16574,
 17218,
 18704,
 20803,
 22751,
 24056,
 24115,
 24363,
 24754,
 25278,
 27704,
 27741,
 27914,
 27985,
 28291,
 31335,
 31423,
 31730,
 32246,
 32616,
 35911,
 36518,
 37524,
 38719,
 39504,
 40769,
 41906,
 42009,
 42439,
 44413,
 44752,
 45529,
 46690,
 49770,
 50686,
 51440,
 54058,
 54323,
 54390,
 56279,
 56659,
 57228,
 58652,
 59957,
 60050,
 64679,
 65329,
 66948,
 67071,
 68161,
 68164,
 70067,
 70668,
 71417,
 71826,
 75000,
 75058,
 75637,
 76136,
 76503,
 76604,
 78743,
 79008,
 79566,
 81410,
 81502,
 81524,
 82173,
 83805,
 84810,
 87157,
 87917,
 88389,
 88469,
 91448,
 92115,
 93167,
 93704,
 96533,
 97320,
 100422,
 100537,
 100722,
 100806,
 101110,
 102245,
 102248,
 102859,
 107221,
 108448,
 108736,
 110078,
 113572,
 113635,
 113896,
 114156,
 116940,
 118319,
 119097,
 120911,
 121139,
 122791,
 124135,
 124856,

Through them we can recover the customers and visually check for their similarity

In [48]:
df.loc[cluster[(1, 1, 5, 13)]]

,New_ID,CustGender,CustomerClassAge,Richness,Expenditure
141,C1010414F27,F,age_3,richness_8,exp_5
803,C1012550F29,F,age_3,richness_8,exp_5
2674,C1018118F28,F,age_3,richness_8,exp_5
4190,C1022681F27,F,age_3,richness_8,exp_5
6203,C1028737F28,F,age_3,richness_8,exp_5
...,...,...,...,...,...
1030101,C9033591F27,F,age_3,richness_8,exp_5
1032215,C9040245F26,F,age_3,richness_8,exp_5
1032891,C9042424F29,F,age_3,richness_8,exp_5
1033931,C9069748F27,F,age_3,richness_8,exp_5


We decided to convert each list of indexes of every keys with the list of the related Customers

In [49]:
for key in cluster.keys():
    
    cluster[key] = df.loc[cluster[key]]['New_ID'].to_list()

Now the dictionary cluster contains as keys the name of the bucket (subvector) and as values the CustomerIDs.

# 1.3 Locality Sensitive Hashing

Now that you prepared your algorithm, it's query time!
We have prepared some dummy users for you to work with.

Download this csv and report the most similar users (comparing them against the dataset provided in Kaggle).
Did your hashing method work properly, what scores have you obtained and how long did it take to run? Provide information and analysis about the results

In [ ]:
query = pd.read_csv("/Users/giacomo/Desktop/ADM_HW4/query_users.csv")

In [ ]:
query

First of all we convert them into class of age, richness and expenditure:

In [ ]:
query['CustomerDOB'] = pd.to_datetime(query['CustomerDOB'])

query['TransactionDate'] = pd.to_datetime(query['TransactionDate']) 

In [ ]:
query.head(5)

In [ ]:
query['CustomerAge'] = 0

In [ ]:
query.loc[query['CustomerDOB'].dt.year != 1800, 'CustomerAge'] = query.loc[query['CustomerDOB'].dt.year != 1800, 'TransactionDate'].dt.year - query.loc[query['CustomerDOB'].dt.year != 1800, 'CustomerDOB'].dt.year 

In [ ]:
query.head(5)

In [ ]:
del query['TransactionDate']

In [ ]:
bins = np.array(list(range(16, 102, 5)))

def age(age):
    
    class_age = np.digitize(age, bins, right=False)  #return the number of the bin
    
    age = 'age_' + str(class_age)
        
    return age

In [ ]:
query['CustomerClassAge'] = query.CustomerAge.apply(lambda x: age(x))

In [ ]:
bin_labels = ['richness_0', 'richness_1', 'richness_2', 'richness_3', 'richness_4', 'richness_5', 'richness_6', 'richness_7', 'richness_8', 'richness_9', 'richness_10']

In [ ]:
query['Richness'] = pd.qcut(query['CustAccountBalance'], q = [0,0.01, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1], labels = bin_labels)

In [ ]:
del query['CustAccountBalance']

In [ ]:
bin_labels = ['exp_1', 'exp_2', 'exp_3', 'exp_4', 'exp_5', 'exp_6', 'exp_7', 'exp_8', 'exp_9', 'exp_10']

In [ ]:
query['Expenditure'] = pd.qcut(query['TransactionAmount (INR)'], q = [0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1], labels = bin_labels)

In [ ]:
del query['TransactionAmount (INR)'], query['CustomerAge']

In [ ]:
query